In [1]:
import random
import re
import numpy as np
import pandas as pd
import nltk

from tqdm import tqdm
import pymorphy3
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.callbacks import CallbackAny2Vec
from corus import load_lenta
from navec import Navec

random_state = 12
random.seed(random_state)
np.random.seed(random_state)

morph = pymorphy3.MorphAnalyzer()
nltk.download('stopwords')
stop_words = stopwords.words('russian')

tqdm.pandas()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\verai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Загрузка, подготовка данных
Загружаем датасет Lenta.Ru-News, оставляем нужные столбцы и применяем случайную выборку для ускорения эксперимента.
Предобработка включает приведение к нижнему регистру, удаление пунктуации, стоп-слов, символов, латинских букв и лемматизацию с помощью pymorphy3.MorphAnalyzer().


In [2]:
#!curl -L https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz -o lenta-ru-news.csv.gz

In [3]:
records = load_lenta('lenta-ru-news.csv.gz')
data = pd.DataFrame(records)
data.columns = ['url', 'title', 'text', 'topic', 'tags', 'date']
data = data[['title', 'text', 'topic']]
data = data.sample(n=100000, random_state=random_state)
data.head()

,title,text,topic
153479,В Афганистане дюжина боевиков ИГ подорвалась н...,"В афганской провинции Нангархар, находящейся н...",Мир
363174,Отток капитала из России с начала года достиг ...,Отток капитала из России по итогам первых четы...,Экономика
306435,"Эдриан Броуди заменит Гари Олдмана в ""Автомоби...",Эдриану Броуди предложили роль в триллере Альб...,Культура
699762,"""Менты"" переехали в Красноярский край",Во вторник в Красноярск прибыла съемочная груп...,Культура
717351,"Шариатский суд пожалел трех иранцев, назначив ...",В четверг три жителя Тегерана были подвергнуты...,Мир


In [4]:
data['topic'].value_counts()

topic
Россия               21908
Мир                  18480
Экономика            10756
Спорт                 8826
Культура              7289
Наука и техника       7161
Бывший СССР           7135
Интернет и СМИ        5926
Из жизни              3697
Дом                   2966
Силовые структуры     2614
Ценности              1058
Бизнес                 959
Путешествия            876
69-я параллель         166
Крым                    84
Культпросвет            48
                        33
Библиотека               9
Легпром                  9
Name: count, dtype: int64

In [5]:
data['topic'].isna().any()

False

In [6]:
def preprocess_text(text):
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление пунктуации, символов и латинских букв — оставляем только русские буквы и пробелы
    text = re.sub(r'[^а-яё\s]', ' ', text)
    # Токенизация и удаление стоп-слов
    tokens = [word for word in text.split() if word not in stop_words]
    # Лемматизация: получение нормальной формы для каждого слова
    tokens = [morph.parse(word)[0].normal_form for word in tokens]
    return " ".join(tokens)

data['combined_text'] = (data['title'] + " " + data['text']).progress_apply(preprocess_text)

100%|██████████| 100000/100000 [10:53<00:00, 153.11it/s]


In [7]:
# Разбиение данных со стратификацией в пропорции 60/20/20
X = data['combined_text']
y = data['topic']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.6, stratify=y, random_state=random_state)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=random_state)
print("Train:", X_train.shape, "Validation:", X_val.shape, "Test:", X_test.shape)

Train: (60000,) Validation: (20000,) Test: (20000,)


### Обучение эмбеддингов Word2Vec

In [8]:
sentences = [text.split() for text in X_train]

In [9]:
class TqdmProgressCallback(CallbackAny2Vec):
    def __init__(self, total_epochs):
        self.total_epochs = total_epochs
        self.pbar = tqdm(total=total_epochs, desc="Epochs", unit="epoch")
        self.epoch = 0

    def on_epoch_end(self, model):
        self.epoch += 1
        self.pbar.update(1)

    def on_train_end(self, model):
        self.pbar.close()

Используем архитектуру skip-gram (sg=1) с параметрами:
 - vector_size=256: Стандартное значение - 100. Увеличение размерности до 256 для хорошего качества векторов.
 - window=7: Обычно используется окно размером 5, но увеличение до 7 расширяет контекст, охватывая больше соседних слов.
 - min_count=10: Стандартное значение зачастую равно 5. Установка min_count на 10 позволяет исключить очень редкие слова,
уменьшая шум в обучении и сокращая размер словаря.

In [10]:
w2v_model = Word2Vec(
    sentences,
    vector_size=256,   # Размерность векторов
    window=7,          # Размер окна контекста
    min_count=10,       # Игнорируем редкие слова
    sg=1,              # Skip-gram модель
    workers=4,
    epochs=20,
    seed=random_state,
    callbacks=[TqdmProgressCallback(total_epochs=20)]
)

Epochs: 100%|██████████| 20/20 [04:23<00:00, 13.17s/epoch]


##### Intrinsic-оценка: методы doesnt_match и most_similar

In [11]:
words_group1 = ["президент", "министр", "кошка", "правительство"]
mismatch1 = w2v_model.wv.doesnt_match(words_group1)
print(f"Doesn't match из {words_group1}: {mismatch1}")

words_group2 = ["машина", "автомобиль", "велосипед", "яблоко"]
mismatch2 = w2v_model.wv.doesnt_match(words_group2)
print(f"Doesn't match из {words_group2}: {mismatch2}")

Doesn't match из ['президент', 'министр', 'кошка', 'правительство']: кошка
Doesn't match из ['машина', 'автомобиль', 'велосипед', 'яблоко']: яблоко


In [12]:
similar_words = w2v_model.wv.most_similar("экономика", topn=5)
print("most_similar для 'экономика':")
for word, score in similar_words:
    print(f"{word}: {score:.3f}")

most_similar для 'экономика':
ввп: 0.637
экономический: 0.633
рецессия: 0.593
финансы: 0.547
конкурентоспособность: 0.540


In [13]:
sim_cat_dog = w2v_model.wv.similarity("кошка", "собака")
print(f"Сходство между 'кошка' и 'собака': {sim_cat_dog:.3f}")

Сходство между 'кошка' и 'собака': 0.547


In [14]:
analogy = w2v_model.wv.most_similar(positive=["король", "женщина"], negative=["мужчина"], topn=5)
print("\nРезультат векторной арифметики (король + женщина - мужчина):")
for word, score in analogy:
    print(f"{word}: {score:.3f}")



Результат векторной арифметики (король + женщина - мужчина):
монарх: 0.440
гьянендра: 0.421
сайрусый: 0.410
фахд: 0.404
майли: 0.404


По точечной оценке работает хорошо, но с арифметикой очевидно не справился)

### Загрузка предобученных эмбеддингов (Navec и RusVectores)

Выбрала navec_news_v1_1B_250K_300d_100q (написано что для новостей, в 2 раза меньше чем `hudlit` и покрывает те же 98% слов в news articles) и ruwikiruscorpora_upos_skipgram_300_2_2018

In [15]:
# navec_news_v1_1B_250K_300d_100q
# !curl -L https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar -o navec_news.tar


In [16]:
# ruwikiruscorpora_upos_skipgram_300_2_2018
# !curl -L -o ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz https://rusvectores.org/static/models/rusvectores4/ruwikiruscorpora/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz

In [17]:
navec = Navec.load("navec_news.tar")
rusvectores = KeyedVectors.load_word2vec_format('ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz')

In [18]:
rusvectores.most_similar(positive=['день_NOUN'], topn=10)

[('неделя_NOUN', 0.7097942233085632),
 ('месяц_NOUN', 0.6908209919929504),
 ('утро_NOUN', 0.6243225932121277),
 ('днемя_NOUN', 0.6224159598350525),
 ('днями_NOUN', 0.6100884675979614),
 ('воскресенье_NOUN', 0.6087555885314941),
 ('вечер_NOUN', 0.6030082702636719),
 ('дня_NOUN', 0.592153012752533),
 ('час_NOUN', 0.590473473072052),
 ('сутки_NOUN', 0.5869504809379578)]

In [19]:
print(morph.parse('день')[0].tag.POS)

NOUN


In [20]:
def get_rusvectores_token(word, model):
    parsed = morph.parse(word)[0]
    pos = parsed.tag.POS 
    if pos:
        token_with_tag = f"{word}_{pos}"
        if token_with_tag in model:
            return token_with_tag
    return word if word in model else None

def get_avg_vector(text, model, vector_size, use_pos_tag=False):
    tokens = text.split()
    vectors = []
    for token in tqdm(tokens, leave=False, desc="Tokens", disable=not use_pos_tag):
        key = get_rusvectores_token(token, model) if use_pos_tag else token
        if key is not None and key in model:
            vectors.append(model[key])
    if vectors:
        return np.mean(vectors, axis=0)
    return np.zeros(vector_size)


In [21]:
# Для обученной модели Word2Vec
X_train_w2v = np.vstack(X_train.progress_apply(lambda x: get_avg_vector(x, w2v_model.wv, w2v_model.vector_size, use_pos_tag=False)))
X_val_w2v   = np.vstack(X_val.progress_apply(lambda x: get_avg_vector(x, w2v_model.wv, w2v_model.vector_size, use_pos_tag=False)))
X_test_w2v  = np.vstack(X_test.progress_apply(lambda x: get_avg_vector(x, w2v_model.wv, w2v_model.vector_size, use_pos_tag=False)))

100%|██████████| 20000/20000 [00:03<00:00, 6618.13it/s]


In [22]:
# Векторизация для navec (без POS-тегов)
X_train_navec = np.vstack(X_train.progress_apply(lambda x: get_avg_vector(x, navec, 300, use_pos_tag=False)))
X_val_navec   = np.vstack(X_val.progress_apply(lambda x: get_avg_vector(x, navec, 300, use_pos_tag=False)))
X_test_navec  = np.vstack(X_test.progress_apply(lambda x: get_avg_vector(x, navec, 300, use_pos_tag=False)))

100%|██████████| 20000/20000 [00:09<00:00, 2000.98it/s]


In [23]:
# Векторизация для rusvectores (с POS-тегированием)
X_train_rusvectores = np.vstack(X_train.progress_apply(lambda x: get_avg_vector(x, rusvectores, 300, use_pos_tag=True)))
X_val_rusvectores   = np.vstack(X_val.progress_apply(lambda x: get_avg_vector(x, rusvectores, 300, use_pos_tag=True)))
X_test_rusvectores  = np.vstack(X_test.progress_apply(lambda x: get_avg_vector(x, rusvectores, 300, use_pos_tag=True)))

100%|██████████| 20000/20000 [03:00<00:00, 111.02it/s]


### Обучение Logistic Regression для разных вариантов векторизации

In [24]:
def train_and_evaluate_lr(X_train, y_train, X_val, y_val, model_name="Model"):
    lr = LogisticRegression(
        random_state=random_state, 
        max_iter=1000)
    lr.fit(X_train, y_train)
    y_val_pred = lr.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    
    print(f"Validation accuracy ({model_name}): {val_accuracy:.4f}")
    print(f"\nОтчет о классификации {model_name}:")
    print(classification_report(y_val.values, y_val_pred, zero_division=0))
    
    return lr, y_val_pred, val_accuracy

In [25]:
# Применение для обученной модели word2vec
lr_w2v, y_val_pred_w2v, acc_w2v = train_and_evaluate_lr(X_train_w2v, y_train, X_val_w2v, y_val, model_name="word2vec")

Validation accuracy (word2vec): 0.7906

Отчет о классификации word2vec:
                   precision    recall  f1-score   support

                        0.00      0.00      0.00         7
   69-я параллель       0.67      0.18      0.29        33
       Библиотека       0.00      0.00      0.00         2
           Бизнес       0.36      0.08      0.13       192
      Бывший СССР       0.80      0.78      0.79      1427
              Дом       0.83      0.79      0.81       593
         Из жизни       0.62      0.55      0.58       740
   Интернет и СМИ       0.75      0.65      0.70      1185
             Крым       1.00      0.06      0.11        17
    Культпросвет        0.00      0.00      0.00        10
         Культура       0.86      0.86      0.86      1458
          Легпром       0.00      0.00      0.00         2
              Мир       0.77      0.84      0.80      3696
  Наука и техника       0.80      0.80      0.80      1432
      Путешествия       0.76      0.57    

In [26]:
# Применение для модели Navec
lr_navec, y_val_pred_navec, acc_navec = train_and_evaluate_lr(X_train_navec, y_train, X_val_navec, y_val, model_name="navec")

Validation accuracy (navec): 0.7810

Отчет о классификации navec:
                   precision    recall  f1-score   support

                        0.00      0.00      0.00         7
   69-я параллель       0.40      0.12      0.19        33
       Библиотека       0.00      0.00      0.00         2
           Бизнес       0.43      0.10      0.17       192
      Бывший СССР       0.79      0.77      0.78      1427
              Дом       0.80      0.79      0.80       593
         Из жизни       0.60      0.54      0.57       740
   Интернет и СМИ       0.73      0.65      0.69      1185
             Крым       1.00      0.06      0.11        17
    Культпросвет        0.00      0.00      0.00        10
         Культура       0.84      0.86      0.85      1458
          Легпром       0.00      0.00      0.00         2
              Мир       0.77      0.82      0.79      3696
  Наука и техника       0.78      0.80      0.79      1432
      Путешествия       0.69      0.49      0.57

In [27]:
# Применение для модели RusVectores
lr_rusvectores, y_val_pred_rusvectores, acc_rusvectores = train_and_evaluate_lr(X_train_rusvectores, y_train, X_val_rusvectores, y_val, model_name="rusvectores")

Validation accuracy (rusvectores): 0.7376

Отчет о классификации rusvectores:
                   precision    recall  f1-score   support

                        0.00      0.00      0.00         7
   69-я параллель       0.00      0.00      0.00        33
       Библиотека       0.00      0.00      0.00         2
           Бизнес       0.00      0.00      0.00       192
      Бывший СССР       0.76      0.56      0.64      1427
              Дом       0.79      0.64      0.71       593
         Из жизни       0.58      0.42      0.49       740
   Интернет и СМИ       0.71      0.58      0.64      1185
             Крым       0.00      0.00      0.00        17
    Культпросвет        0.00      0.00      0.00        10
         Культура       0.81      0.84      0.82      1458
          Легпром       0.00      0.00      0.00         2
              Мир       0.72      0.82      0.77      3696
  Наука и техника       0.74      0.75      0.75      1432
      Путешествия       0.58      0.

Лучше всего показала себя обученная word2vec, что логично, так как, несмотря на разделение на выборки трейн/валидация/тест, обучена на одном корпусе.

### TF-IDF взвешенное усреднение эмбеддингов

In [28]:
tfidf_vect = TfidfVectorizer(tokenizer=lambda x: x.split(), lowercase=False)
tfidf_vect.fit(X_train)

c:\Users\verai\projects\AITH_dl_nlp\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(lowercase=False,
                tokenizer=<function <lambda> at 0x000001A94CD91DA0>)

TF-IDF позволяет учитывать важность слова в документе. Взвешиваем word2vec эмбеддинги с использованием коэффициентов idf.  Используем именно idf, так как частота слова уже учтена при усреднении, когда каждое появление слова добавляет свой вклад в итоговый вектор.

In [29]:
def get_tfidf_weighted_avg_vector(text, embedding_model, vector_size, tfidf_vectorizer):
    tokens = text.split()
    weighted_vec = np.zeros(vector_size)
    weight_sum = 0.0
    for token in tokens:
        if token in embedding_model and token in tfidf_vectorizer.vocabulary_:
            idx = tfidf_vectorizer.vocabulary_[token]
            weight = tfidf_vectorizer.idf_[idx]
            weighted_vec += embedding_model[token] * weight
            weight_sum += weight
    if weight_sum != 0:
        weighted_vec /= weight_sum
    return weighted_vec

In [30]:
# Применяем TF-IDF взвешивание для лучшей модели (word2vec)
X_train_tfidf = np.vstack(X_train.progress_apply(lambda x: get_tfidf_weighted_avg_vector(x, w2v_model.wv, w2v_model.vector_size, tfidf_vect)))
X_val_tfidf   = np.vstack(X_val.progress_apply(lambda x: get_tfidf_weighted_avg_vector(x, w2v_model.wv, w2v_model.vector_size, tfidf_vect)))
X_test_tfidf  = np.vstack(X_test.progress_apply(lambda x: get_tfidf_weighted_avg_vector(x, w2v_model.wv, w2v_model.vector_size, tfidf_vect)))

100%|██████████| 20000/20000 [00:07<00:00, 2800.02it/s]


In [31]:
print("Оценка модели с TF-IDF взвешенным усреднением эмбеддингов (word2vec):")
lr_tfidf, y_val_pred_tfidf, acc_tfidf = train_and_evaluate_lr(
    X_train_tfidf, y_train, X_val_tfidf, y_val, model_name="word2vec_tfidf"
)

Оценка модели с TF-IDF взвешенным усреднением эмбеддингов (word2vec):
Validation accuracy (word2vec_tfidf): 0.7862

Отчет о классификации word2vec_tfidf:
                   precision    recall  f1-score   support

                        0.00      0.00      0.00         7
   69-я параллель       0.70      0.21      0.33        33
       Библиотека       0.00      0.00      0.00         2
           Бизнес       0.44      0.07      0.12       192
      Бывший СССР       0.80      0.78      0.79      1427
              Дом       0.84      0.78      0.80       593
         Из жизни       0.60      0.55      0.58       740
   Интернет и СМИ       0.73      0.65      0.69      1185
             Крым       1.00      0.12      0.21        17
    Культпросвет        0.00      0.00      0.00        10
         Культура       0.85      0.87      0.86      1458
          Легпром       0.00      0.00      0.00         2
              Мир       0.76      0.83      0.79      3696
  Наука и техника  

### Финальное сравнение моделей на тестовой выборке
Выводятся тестовая точность и отчеты о классификации для всех моделей.

In [32]:
# Модель word2vec
y_test_pred_w2v = lr_w2v.predict(X_test_w2v)
acc_test_w2v = accuracy_score(y_test, y_test_pred_w2v)
print("Тестовая точность (word2vec): {:.4f}".format(acc_test_w2v))
print(classification_report(y_test.values, y_test_pred_w2v, zero_division=0))

Тестовая точность (word2vec): 0.7919
                   precision    recall  f1-score   support

                        0.00      0.00      0.00         6
   69-я параллель       1.00      0.15      0.26        33
       Библиотека       0.00      0.00      0.00         2
           Бизнес       0.69      0.11      0.20       192
      Бывший СССР       0.80      0.78      0.79      1427
              Дом       0.86      0.76      0.81       593
         Из жизни       0.62      0.57      0.60       739
   Интернет и СМИ       0.73      0.67      0.70      1185
             Крым       0.00      0.00      0.00        17
    Культпросвет        0.00      0.00      0.00         9
         Культура       0.86      0.88      0.87      1458
          Легпром       0.00      0.00      0.00         2
              Мир       0.78      0.84      0.81      3696
  Наука и техника       0.78      0.83      0.80      1433
      Путешествия       0.78      0.54      0.64       175
           Россия 

In [33]:
# Модель Navec
y_test_pred_navec = lr_navec.predict(X_test_navec)
acc_test_navec = accuracy_score(y_test, y_test_pred_navec)
print("Тестовая точность (navec): {:.4f}".format(acc_test_navec))
print(classification_report(y_test.values, y_test_pred_navec, zero_division=0))

Тестовая точность (navec): 0.7801
                   precision    recall  f1-score   support

                        0.00      0.00      0.00         6
   69-я параллель       0.62      0.15      0.24        33
       Библиотека       0.00      0.00      0.00         2
           Бизнес       0.57      0.11      0.18       192
      Бывший СССР       0.79      0.78      0.78      1427
              Дом       0.82      0.75      0.78       593
         Из жизни       0.59      0.56      0.57       739
   Интернет и СМИ       0.71      0.66      0.68      1185
             Крым       0.00      0.00      0.00        17
    Культпросвет        0.00      0.00      0.00         9
         Культура       0.85      0.85      0.85      1458
          Легпром       0.00      0.00      0.00         2
              Мир       0.77      0.82      0.80      3696
  Наука и техника       0.76      0.80      0.78      1433
      Путешествия       0.69      0.49      0.57       175
           Россия    

In [34]:
# Модель RusVectores 
y_test_pred_rusvectores = lr_rusvectores.predict(X_test_rusvectores)
acc_test_rusvectores = accuracy_score(y_test, y_test_pred_rusvectores)
print("Тестовая точность (rusvectores): {:.4f}".format(acc_test_rusvectores))
print(classification_report(y_test.values, y_test_pred_rusvectores, zero_division=0))

Тестовая точность (rusvectores): 0.7406
                   precision    recall  f1-score   support

                        0.00      0.00      0.00         6
   69-я параллель       0.00      0.00      0.00        33
       Библиотека       0.00      0.00      0.00         2
           Бизнес       0.00      0.00      0.00       192
      Бывший СССР       0.78      0.57      0.66      1427
              Дом       0.82      0.64      0.72       593
         Из жизни       0.60      0.44      0.51       739
   Интернет и СМИ       0.70      0.57      0.63      1185
             Крым       0.00      0.00      0.00        17
    Культпросвет        0.00      0.00      0.00         9
         Культура       0.81      0.85      0.83      1458
          Легпром       0.00      0.00      0.00         2
              Мир       0.72      0.82      0.77      3696
  Наука и техника       0.73      0.78      0.76      1433
      Путешествия       0.78      0.12      0.21       175
           Росс

In [35]:
# Модель с TF-IDF взвешиванием (обученный word2vec)
y_test_pred_tfidf = lr_tfidf.predict(X_test_tfidf)
acc_test_tfidf = accuracy_score(y_test, y_test_pred_tfidf)
print("Тестовая точность (word2vec с TF-IDF): {:.4f}".format(acc_test_tfidf))
print(classification_report(y_test.values, y_test_pred_tfidf, zero_division=0))

Тестовая точность (word2vec с TF-IDF): 0.7887
                   precision    recall  f1-score   support

                        0.00      0.00      0.00         6
   69-я параллель       0.70      0.21      0.33        33
       Библиотека       0.00      0.00      0.00         2
           Бизнес       0.75      0.11      0.19       192
      Бывший СССР       0.80      0.78      0.79      1427
              Дом       0.85      0.76      0.80       593
         Из жизни       0.60      0.57      0.58       739
   Интернет и СМИ       0.73      0.66      0.69      1185
             Крым       0.00      0.00      0.00        17
    Культпросвет        0.00      0.00      0.00         9
         Культура       0.86      0.88      0.87      1458
          Легпром       0.00      0.00      0.00         2
              Мир       0.78      0.84      0.81      3696
  Наука и техника       0.78      0.82      0.80      1433
      Путешествия       0.76      0.52      0.62       175
         

| Модель                    | Тестовая точность | Macro Avg F1 | Weighted Avg F1 |
|---------------------------|-------------------|--------------|-----------------|
| word2vec с TF-IDF         | 0.7880            | 0.50         | 0.78            |
| rusvectores               | 0.7403            | 0.41         | 0.72            |
| navec                     | 0.7799            | 0.49         | 0.77            |
| word2vec (без TF-IDF)     | 0.7917            | 0.50         | 0.78            |

Лучшей моделью эмбеддингов оказалась word2vec (без TF-IDF), так как она продемонстрировала выше тестовую точность (0.7917) по сравнению с другими вариантами, при схожих показателях Macro Avg F1 и Weighted Avg F1.